In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from tqdm import tqdm
from sklearn import preprocessing
import tensorflow as tf
from statsmodels.tsa.arima.model import ARIMA, ARIMAResults
from statsmodels.tsa.seasonal import seasonal_decompose
import pmdarima
import traceback
import logging
import warnings
import folium
from folium import plugins

In [2]:
def error(y_pred, y_true):
    return (np.abs(y_pred - y_true)) / y_pred

In [3]:
df = pd.read_csv('train_data.csv')
df

,timestamp,lon,lat,likescount,commentscount,symbols_cnt,words_cnt,hashtags_cnt,mentions_cnt,links_cnt,emoji_cnt,point
0,1565607600,30.329370,59.940488,63,4,52,6,0,0,0,0,0101000020E6100000DBC1F19351543E4006FC5DE561F8...
1,1565568000,30.342845,59.869542,44,0,40,0,4,0,0,0,0101000020E6100000544263A8C4573E40C27FCF284DEF...
2,1565568000,30.322632,60.039332,30,0,77,7,0,0,0,1,0101000020E61000009E01B90998523E40888919D70805...
3,1565568000,30.322632,59.938238,13,0,634,17,32,0,0,7,0101000020E61000009E01B90998523E40CEB8CE2B18F8...
4,1565568000,30.309158,59.932612,8,0,0,0,0,0,0,0,0101000020E6100000268147F5244F3E40494A4ED65FF7...
...,...,...,...,...,...,...,...,...,...,...,...,...
8526521,1565978400,30.324878,59.932612,48,0,15,2,0,0,0,1,0101000020E61000005DECCB372B533E40494A4ED65FF7...
8526522,1565978400,30.324878,59.932612,20,1,0,0,0,0,0,0,0101000020E61000005DECCB372B533E40494A4ED65FF7...
8526523,1565978400,30.324878,59.932612,54,2,75,7,0,0,0,6,0101000020E61000005DECCB372B533E40494A4ED65FF7...
8526524,1565971200,30.324878,59.932612,15,0,350,4,25,0,0,0,0101000020E61000005DECCB372B533E40494A4ED65FF7...


In [4]:
#сократим количество потребляемой оперативы
df = df[['timestamp', 'point']]
df

,timestamp,point
0,1565607600,0101000020E6100000DBC1F19351543E4006FC5DE561F8...
1,1565568000,0101000020E6100000544263A8C4573E40C27FCF284DEF...
2,1565568000,0101000020E61000009E01B90998523E40888919D70805...
3,1565568000,0101000020E61000009E01B90998523E40CEB8CE2B18F8...
4,1565568000,0101000020E6100000268147F5244F3E40494A4ED65FF7...
...,...,...
8526521,1565978400,0101000020E61000005DECCB372B533E40494A4ED65FF7...
8526522,1565978400,0101000020E61000005DECCB372B533E40494A4ED65FF7...
8526523,1565978400,0101000020E61000005DECCB372B533E40494A4ED65FF7...
8526524,1565971200,0101000020E61000005DECCB372B533E40494A4ED65FF7...


In [5]:
len(df.point.unique())

7041

In [6]:
# validation contains 710 cubes
# February 2020
valid_df = pd.read_csv('valid.csv')
valid_df = valid_df[['point', 'hour', 'sum', 'error', 'lon', 'lat']]
valid_df['timestamp']= [datetime.fromtimestamp(x) for x in valid_df['hour']]
valid_df['Dates'] = pd.to_datetime(valid_df['timestamp']).dt.date
valid_df['Time'] = pd.to_datetime(valid_df['timestamp']).dt.time
valid_df.sort_values(by=['sum'], ascending=False)

,point,hour,sum,error,lon,lat,timestamp,Dates,Time
504,0101000020E610000091029C327E593E40494A4ED65FF7...,1580587200,40,38.754735,30.349582,59.932612,2020-02-01 23:00:00,2020-02-01,23:00:00
638,0101000020E61000004D98FA185E5C3E40D4F7F058CCF6...,1582560000,39,3.629735,30.360811,59.928111,2020-02-24 19:00:00,2020-02-24,19:00:00
100,0101000020E6100000A4566D514B503E4006FC5DE561F8...,1581094800,38,17.754735,30.313649,59.940488,2020-02-07 20:00:00,2020-02-07,20:00:00
529,0101000020E6100000A4566D514B503E4006FC5DE561F8...,1580835600,38,36.754735,30.313649,59.940488,2020-02-04 20:00:00,2020-02-04,20:00:00
234,0101000020E6100000669634C7914E3E407AB3D8B484F7...,1581019200,37,4.921402,30.306912,59.933737,2020-02-06 23:00:00,2020-02-06,23:00:00
...,...,...,...,...,...,...,...,...,...
79,0101000020E61000000FD8C18EA45A3E408991431816F7...,1580666400,5,0.245265,30.354073,59.930362,2020-02-02 21:00:00,2020-02-02,21:00:00
506,0101000020E6100000268147F5244F3E40DB251193A9F7...,1581001200,5,3.754735,30.309158,59.934863,2020-02-06 18:00:00,2020-02-06,18:00:00
507,0101000020E61000006141807FDE503E402B348A98ADF9...,1580785200,5,3.754735,30.315895,59.950610,2020-02-04 06:00:00,2020-02-04,06:00:00
510,0101000020E61000004D98FA185E5C3E40CAE971F73AF7...,1580882400,5,0.004735,30.360811,59.931487,2020-02-05 09:00:00,2020-02-05,09:00:00


In [7]:
len(valid_df.point.unique())

155

In [8]:
true_point = sorted(valid_df.point.unique())
true_point

['0101000020E610000000000000000000000000000000000000',
 '0101000020E61000000C830D47F15C3E400A41C338F1F64D40',
 '0101000020E61000000C830D47F15C3E40CAE971F73AF74D40',
 '0101000020E61000000FD8C18EA45A3E408991431816F74D40',
 '0101000020E61000000FD8C18EA45A3E40CAE971F73AF74D40',
 '0101000020E6100000132D76D657583E408991431816F74D40',
 '0101000020E61000001868CE7F1F343E40975BBA5723FE4D40',
 '0101000020E6100000199C86BCF6773E401FBF4913EFF54D40',
 '0101000020E6100000199C86BCF6773E406A1274D638F64D40',
 '0101000020E61000001CD7DE65BE533E40A554BF083DF84D40',
 '0101000020E61000002012370F862F3E40A08C1570AEEA4D40',
 '0101000020E6100000202C93AD71513E4009249E9F54F24D40',
 '0101000020E6100000202C93AD71513E401E66D608FBFB4D40',
 '0101000020E6100000202C93AD71513E4041010E3DD6EB4D40',
 '0101000020E6100000202C93AD71513E4054F844A221FD4D40',
 '0101000020E6100000202C93AD71513E4070B5CC78A7F64D40',
 '0101000020E6100000202C93AD71513E407A441D895BF54D40',
 '0101000020E6100000202C93AD71513E40CEB8CE2B18F84D40',
 '01010000

In [9]:
# Оставим в датасете только интересующие нас кубы и переведем timestamp в даты
df['timestamp']= [datetime.fromtimestamp(x) for x in df['timestamp']]
df_clear = df[df.point.isin(true_point)]
df_clear

C:\Users\gto_n\AppData\Local\Temp\ipykernel_12112\1137194191.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['timestamp']= [datetime.fromtimestamp(x) for x in df['timestamp']]


,timestamp,point
0,2019-08-12 14:00:00,0101000020E6100000DBC1F19351543E4006FC5DE561F8...
1,2019-08-12 03:00:00,0101000020E6100000544263A8C4573E40C27FCF284DEF...
2,2019-08-12 03:00:00,0101000020E61000009E01B90998523E40888919D70805...
3,2019-08-12 03:00:00,0101000020E61000009E01B90998523E40CEB8CE2B18F8...
4,2019-08-12 03:00:00,0101000020E6100000268147F5244F3E40494A4ED65FF7...
...,...,...
8526521,2019-08-16 21:00:00,0101000020E61000005DECCB372B533E40494A4ED65FF7...
8526522,2019-08-16 21:00:00,0101000020E61000005DECCB372B533E40494A4ED65FF7...
8526523,2019-08-16 21:00:00,0101000020E61000005DECCB372B533E40494A4ED65FF7...
8526524,2019-08-16 19:00:00,0101000020E61000005DECCB372B533E40494A4ED65FF7...


In [10]:
df_clear

,timestamp,point
0,2019-08-12 14:00:00,0101000020E6100000DBC1F19351543E4006FC5DE561F8...
1,2019-08-12 03:00:00,0101000020E6100000544263A8C4573E40C27FCF284DEF...
2,2019-08-12 03:00:00,0101000020E61000009E01B90998523E40888919D70805...
3,2019-08-12 03:00:00,0101000020E61000009E01B90998523E40CEB8CE2B18F8...
4,2019-08-12 03:00:00,0101000020E6100000268147F5244F3E40494A4ED65FF7...
...,...,...
8526521,2019-08-16 21:00:00,0101000020E61000005DECCB372B533E40494A4ED65FF7...
8526522,2019-08-16 21:00:00,0101000020E61000005DECCB372B533E40494A4ED65FF7...
8526523,2019-08-16 21:00:00,0101000020E61000005DECCB372B533E40494A4ED65FF7...
8526524,2019-08-16 19:00:00,0101000020E61000005DECCB372B533E40494A4ED65FF7...


In [11]:

working_df = df_clear.groupby(['point', 'timestamp']).size().reset_index(name='counts')
working_df.sort_values(by=['counts'], ascending=False)

,point,timestamp,counts
229619,0101000020E61000004D98FA185E5C3E40CAE971F73AF7...,2020-01-20 02:00:00,329
309069,0101000020E61000006141807FDE503E402B348A98ADF9...,2019-05-09 22:00:00,253
43153,0101000020E6100000199C86BCF6773E401FBF4913EFF5...,2019-02-05 00:00:00,248
309070,0101000020E61000006141807FDE503E402B348A98ADF9...,2019-05-09 23:00:00,230
327447,0101000020E61000006141807FDE503E408F0FCDBE88F9...,2020-01-19 20:00:00,200
...,...,...,...
657347,0101000020E6100000BE54A7FF7E423E40F91CC68469FC...,2019-12-11 14:00:00,1
657346,0101000020E6100000BE54A7FF7E423E40F91CC68469FC...,2019-12-11 11:00:00,1
657345,0101000020E6100000BE54A7FF7E423E40F91CC68469FC...,2019-12-11 08:00:00,1
657344,0101000020E6100000BE54A7FF7E423E40F91CC68469FC...,2019-12-11 02:00:00,1


In [12]:
working_df = df_clear.groupby(['point','timestamp']).size().unstack(fill_value=
                                                         0
                                                         ).stack()
working_df = working_df.reset_index(name='counts')
working_df

,point,timestamp,counts
0,0101000020E61000000000000000000000000000000000...,2019-01-01 03:00:00,3
1,0101000020E61000000000000000000000000000000000...,2019-01-01 04:00:00,2
2,0101000020E61000000000000000000000000000000000...,2019-01-01 05:00:00,2
3,0101000020E61000000000000000000000000000000000...,2019-01-01 06:00:00,0
4,0101000020E61000000000000000000000000000000000...,2019-01-01 07:00:00,0
...,...,...,...
1473115,0101000020E6100000F92E3C2824663E40134E7A739601...,2020-01-31 22:00:00,1
1473116,0101000020E6100000F92E3C2824663E40134E7A739601...,2020-01-31 23:00:00,0
1473117,0101000020E6100000F92E3C2824663E40134E7A739601...,2020-02-01 00:00:00,2
1473118,0101000020E6100000F92E3C2824663E40134E7A739601...,2020-02-01 01:00:00,0


In [13]:
# Разделим timestamp на дату и время
working_df['Dates'] = pd.to_datetime(working_df['timestamp']).dt.date
working_df['Time'] = pd.to_datetime(working_df['timestamp']).dt.time
working_df

,point,timestamp,counts,Dates,Time
0,0101000020E61000000000000000000000000000000000...,2019-01-01 03:00:00,3,2019-01-01,03:00:00
1,0101000020E61000000000000000000000000000000000...,2019-01-01 04:00:00,2,2019-01-01,04:00:00
2,0101000020E61000000000000000000000000000000000...,2019-01-01 05:00:00,2,2019-01-01,05:00:00
3,0101000020E61000000000000000000000000000000000...,2019-01-01 06:00:00,0,2019-01-01,06:00:00
4,0101000020E61000000000000000000000000000000000...,2019-01-01 07:00:00,0,2019-01-01,07:00:00
...,...,...,...,...,...
1473115,0101000020E6100000F92E3C2824663E40134E7A739601...,2020-01-31 22:00:00,1,2020-01-31,22:00:00
1473116,0101000020E6100000F92E3C2824663E40134E7A739601...,2020-01-31 23:00:00,0,2020-01-31,23:00:00
1473117,0101000020E6100000F92E3C2824663E40134E7A739601...,2020-02-01 00:00:00,2,2020-02-01,00:00:00
1473118,0101000020E6100000F92E3C2824663E40134E7A739601...,2020-02-01 01:00:00,0,2020-02-01,01:00:00


In [14]:
working_df.timestamp.unique()

array(['2019-01-01T03:00:00.000000000', '2019-01-01T04:00:00.000000000',
       '2019-01-01T05:00:00.000000000', ...,
       '2020-02-01T00:00:00.000000000', '2020-02-01T01:00:00.000000000',
       '2020-02-01T02:00:00.000000000'], dtype='datetime64[ns]')

In [15]:
points = working_df.point.unique()
points

array(['0101000020E610000000000000000000000000000000000000',
       '0101000020E61000000C830D47F15C3E400A41C338F1F64D40',
       '0101000020E61000000C830D47F15C3E40CAE971F73AF74D40',
       '0101000020E61000000FD8C18EA45A3E408991431816F74D40',
       '0101000020E61000000FD8C18EA45A3E40CAE971F73AF74D40',
       '0101000020E6100000132D76D657583E408991431816F74D40',
       '0101000020E61000001868CE7F1F343E40975BBA5723FE4D40',
       '0101000020E6100000199C86BCF6773E401FBF4913EFF54D40',
       '0101000020E6100000199C86BCF6773E406A1274D638F64D40',
       '0101000020E61000001CD7DE65BE533E40A554BF083DF84D40',
       '0101000020E61000002012370F862F3E40A08C1570AEEA4D40',
       '0101000020E6100000202C93AD71513E4009249E9F54F24D40',
       '0101000020E6100000202C93AD71513E401E66D608FBFB4D40',
       '0101000020E6100000202C93AD71513E4041010E3DD6EB4D40',
       '0101000020E6100000202C93AD71513E4054F844A221FD4D40',
       '0101000020E6100000202C93AD71513E4070B5CC78A7F64D40',
       '0101000020E61000

In [16]:
len(points)

155

In [17]:
import warnings
warnings.filterwarnings('ignore')
predict_df = pd.DataFrame(columns=['point', 'hour', 'sum', 'error', 'timestamp'])
for point in tqdm(points):
    example_train = working_df[working_df.point == point]
    example_train.sort_values(by=['timestamp'], inplace=True, ascending=True)
    forecasting_range = pd.period_range(start=pd.to_datetime(example_train['timestamp'].iloc[-1]), end='2020-03-1 23:00:00', freq='1H')
    model = pmdarima.arima.ARIMA(order=(2, 0, 2), seasonal_order=(2, 0, 2, 7))
    model.fit(example_train['counts'])
    forecast= model.predict(n_periods=len(forecasting_range), return_conf_int=False)
    tmp = pd.DataFrame(columns=['point', 'hour', 'sum', 'error', 'timestamp'])
    tmp['timestamp'] = forecasting_range.to_timestamp()
    # tmp['hour'] = forecasting_range.to_timestamp()
    tmp['sum'] = forecast.values
    tmp['point'] = point
    tmp['hour'] = (forecasting_range.to_timestamp() - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
    example_test = valid_df[valid_df.point == point]
    example_test.sort_values(by=['timestamp'], inplace=True)
    tmp = tmp[tmp.timestamp.isin(example_test.timestamp)]
    tmp['error'] = error(tmp['sum'].values, example_test['sum'].values)
    predict_df = pd.concat([predict_df, tmp], ignore_index=True)

100%|██████████| 155/155 [1:30:42<00:00, 35.11s/it]


In [18]:
predict_df

,point,hour,sum,error,timestamp
0,0101000020E61000000000000000000000000000000000...,1580810400,7.408419,0.349816,2020-02-04 10:00:00
1,0101000020E61000000000000000000000000000000000...,1580832000,10.281830,0.361625,2020-02-04 16:00:00
2,0101000020E61000000000000000000000000000000000...,1580918400,10.644901,0.127300,2020-02-05 16:00:00
3,0101000020E61000000000000000000000000000000000...,1580947200,7.359219,0.184696,2020-02-06 00:00:00
4,0101000020E61000000000000000000000000000000000...,1581166800,8.843495,0.434613,2020-02-08 13:00:00
...,...,...,...,...,...
704,0101000020E6100000F914E08938443E407916A554F5F8...,1582117200,1.948768,6.184025,2020-02-19 13:00:00
705,0101000020E6100000F914E08938443E407916A554F5F8...,1582743600,2.051917,2.898793,2020-02-26 19:00:00
706,0101000020E6100000F914E08938443E407916A554F5F8...,1582912800,2.033792,1.458462,2020-02-28 18:00:00
707,0101000020E6100000F914E08938443E407916A554F5F8...,1582984800,1.909654,2.141930,2020-02-29 14:00:00


In [20]:
predict_df.error.mean()

1.9504225408042355

In [22]:
predict_df.to_csv('predict.csv')

In [32]:
warnings.filterwarnings('ignore')
predict_df_acc = pd.DataFrame(columns=['point', 'hour', 'sum', 'error', 'timestamp'])
for point in tqdm(points):
    example_train = working_df[working_df.point == point]
    example_train.sort_values(by=['timestamp'], inplace=True, ascending=True)
    forecasting_range = pd.period_range(start=pd.to_datetime(example_train['timestamp'].iloc[-1]), end='2020-03-1 23:00:00', freq='1H')
    model = pmdarima.arima.ARIMA(order=(2, 0, 2), seasonal_order=(2, 0, 2, 24))

    try:
        model.fit(example_train['counts'])
    except np.linalg.LinAlgError:
        print('LinAlgError. Try 7 order')
        model = pmdarima.arima.ARIMA(order=(2, 0, 2), seasonal_order=(2, 0, 2, 7))
        model.fit(example_train['counts'])

    forecast= model.predict(n_periods=len(forecasting_range), return_conf_int=False)
    tmp = pd.DataFrame(columns=['point', 'hour', 'sum', 'error', 'timestamp'])
    tmp['timestamp'] = forecasting_range.to_timestamp()
    # tmp['hour'] = forecasting_range.to_timestamp()
    tmp['sum'] = forecast.values
    tmp['point'] = point
    tmp['hour'] = (forecasting_range.to_timestamp() - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
    example_test = valid_df[valid_df.point == point]
    example_test.sort_values(by=['timestamp'], inplace=True)
    tmp = tmp[tmp.timestamp.isin(example_test.timestamp)]
    tmp['error'] = error(tmp['sum'].values, example_test['sum'].values)
    predict_df_acc = pd.concat([predict_df_acc, tmp], ignore_index=True)

 81%|████████  | 125/155 [13:47:29<3:18:35, 397.19s/it]


KeyboardInterrupt: 

In [33]:
predict_df_acc.error.mean()

1.9501086558795824

In [34]:
predict_df_acc.to_csv('predict_df_acc.csv')

In [17]:
test = pd.read_csv('test.csv')
test = test[['point', 'hour', 'sum', 'error']]
test['timestamp']= [datetime.fromtimestamp(x) for x in test['hour']]
test['Dates'] = pd.to_datetime(test['timestamp']).dt.date
test['Time'] = pd.to_datetime(test['timestamp']).dt.time
test.sort_values(by=['sum'], ascending=False)

,point,hour,sum,error,timestamp,Dates,Time
258,0101000020E6100000C1A95B4732403E40494A4ED65FF7...,1582999200,40,2.446153,2020-02-29 21:00:00,2020-02-29,21:00:00
143,0101000020E6100000A4566D514B503E4006FC5DE561F8...,1581775200,40,38.754735,2020-02-15 17:00:00,2020-02-15,17:00:00
34,0101000020E61000002D2BB0848B4A3E402B348A98ADF9...,1581069600,38,17.754735,2020-02-07 13:00:00,2020-02-07,13:00:00
436,0101000020E6100000C5FE0F8FE53D3E4033438EB75DF6...,1583006400,37,2.865846,2020-02-29 23:00:00,2020-02-29,23:00:00
438,0101000020E6100000328064CC3E483E400C288C4EF3F7...,1581426000,37,35.754735,2020-02-11 16:00:00,2020-02-11,16:00:00
...,...,...,...,...,...,...,...
96,0101000020E6100000202C93AD71513E40CEB8CE2B18F8...,1582441200,5,1.254735,2020-02-23 10:00:00,2020-02-23,10:00:00
404,0101000020E61000008EC74389B67D3E4033438EB75DF6...,1580803200,5,3.754735,2020-02-04 11:00:00,2020-02-04,11:00:00
93,0101000020E6100000CFC2D4BC375B3E408A05A0FCDCFF...,1581249600,5,1.254735,2020-02-09 15:00:00,2020-02-09,15:00:00
570,0101000020E6100000E56B5A23B84F3E40134E7A739601...,1581447600,5,3.754735,2020-02-11 22:00:00,2020-02-11,22:00:00


In [26]:
warnings.filterwarnings('ignore')
predict_df_acc = pd.DataFrame(columns=['point', 'hour', 'sum', 'error', 'timestamp'])
for point in tqdm(points):
    example_train = working_df[working_df.point == point]
    example_train.sort_values(by=['timestamp'], inplace=True, ascending=True)
    forecasting_range = pd.period_range(start=pd.to_datetime(example_train['timestamp'].iloc[-1]), end='2020-03-1 23:00:00', freq='1H')
    model = pmdarima.arima.ARIMA(order=(2, 0, 2), seasonal_order=(2, 0, 2, 24))

    try:
        model.fit(example_train['counts'])
    except np.linalg.LinAlgError:
        print('LinAlgError. Try 7 order')
        model = pmdarima.arima.ARIMA(order=(2, 0, 2), seasonal_order=(2, 0, 2, 7))
        model.fit(example_train['counts'])

    forecast= model.predict(n_periods=len(forecasting_range), return_conf_int=False)
    tmp = pd.DataFrame(columns=['point', 'hour', 'sum', 'error', 'timestamp'])
    tmp['timestamp'] = forecasting_range.to_timestamp()
    # tmp['hour'] = forecasting_range.to_timestamp()
    tmp['sum'] = forecast.values
    tmp['point'] = point
    tmp['hour'] = (forecasting_range.to_timestamp() - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
    example_test = test[test.point == point]
    example_test.sort_values(by=['timestamp'], inplace=True)
    tmp = tmp[tmp.timestamp.isin(example_test.timestamp)]
    tmp['error'] = error(tmp['sum'].values, example_test['sum'].values)
    predict_df_acc = pd.concat([predict_df_acc, tmp], ignore_index=True)

100%|██████████| 155/155 [16:27:16<00:00, 382.17s/it]  


In [27]:
predict_df_acc.error.mean()

0.9578291067281066

In [28]:
predict_df_acc.to_csv('predict_df_test.csv')

In [ ]:
warnings.filterwarnings('ignore')
predict_df_acc = pd.DataFrame(columns=['point', 'hour', 'sum', 'error', 'timestamp'])
for point in tqdm(points):
    example_train = working_df[working_df.point == point]
    example_train.sort_values(by=['timestamp'], inplace=True, ascending=True)
    forecasting_range = pd.period_range(start=pd.to_datetime(example_train['timestamp'].iloc[-1]), end='2020-03-1 23:00:00', freq='1H')

    try:
        model = pmdarima.arima.auto_arima(example_train['counts'], m=30)
    except np.linalg.LinAlgError:
        print('LinAlgError. Try 7 order')
        model = pmdarima.arima.auto_arima(example_train['counts'], m=7)

    forecast= model.predict(n_periods=len(forecasting_range), return_conf_int=False)
    tmp = pd.DataFrame(columns=['point', 'hour', 'sum', 'error', 'timestamp'])
    tmp['timestamp'] = forecasting_range.to_timestamp()
    # tmp['hour'] = forecasting_range.to_timestamp()
    tmp['sum'] = forecast.values
    tmp['point'] = point
    tmp['hour'] = (forecasting_range.to_timestamp() - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
    example_test = test[test.point == point]
    example_test.sort_values(by=['timestamp'], inplace=True)
    tmp = tmp[tmp.timestamp.isin(example_test.timestamp)]
    tmp['error'] = error(tmp['sum'].values, example_test['sum'].values)
    predict_df_acc = pd.concat([predict_df_acc, tmp], ignore_index=True)

  0%|          | 0/155 [00:00<?, ?it/s]

In [91]:
df_clear[df_clear.timestamp == '2020-01-20 02:00:00'][['lat', 'lon']].values

array([[59.92811119, 30.34733609],
       [59.9191073 , 30.33835293],
       [59.9449869 , 30.36530239],
       [59.9551082 , 30.31140347],
       [59.93598759, 30.3248782 ],
       [59.93261222, 30.34958187],
       [59.83344842, 30.34958187],
       [59.93373738, 30.3069119 ],
       [59.93373738, 30.3069119 ],
       [59.93373738, 30.3069119 ],
       [59.92811119, 30.36081082],
       [59.93148702, 30.36081082],
       [59.93148702, 30.36081082],
       [59.93148702, 30.36081082],
       [59.93148702, 30.36081082],
       [59.93148702, 30.36081082],
       [59.93148702, 30.36081082],
       [59.93148702, 30.36081082],
       [59.93148702, 30.36081082],
       [59.93373738, 30.3069119 ],
       [59.93148702, 30.36081082],
       [59.93373738, 30.3069119 ],
       [59.93148702, 30.36081082],
       [59.94948564, 30.30017453],
       [59.93148702, 30.36081082],
       [59.93148702, 30.36081082],
       [59.93148702, 30.36081082],
       [59.93148702, 30.36081082],
       [59.94948564,

In [99]:
from folium.plugins import HeatMap
m = folium.Map(location=[59.9386, 30.3141], zoom_start=13)
stationArr = df_clear[df_clear.timestamp == '2020-01-20 02:00:00'][['lat', 'lon']].values
m.add_child(plugins.HeatMap(stationArr, radius=45))
# HeatMap(stationArr).add_to(folium.FeatureGroup(name='Heat Map').add_to(m))
# folium.LayerControl().add_to(m)

In [95]:
m